# Summary

# Imports

In [ ]:
from pathlib import Path

import pyarrow as pa
import pyarrow.parquet as pq

# Parameters

In [ ]:
NOTEBOOK_PATH = Path('demo_datasets')
NOTEBOOK_PATH.mkdir(exist_ok=True)
NOTEBOOK_PATH

# Datasets

## `demo_dataset_1`

In [ ]:
DATASET_PATH = NOTEBOOK_PATH.joinpath("demo_dataset_1")
DATASET_PATH.mkdir(exist_ok=True)
DATASET_PATH

### Model state

In [ ]:
!cp train_neural_network/train_classifier-permute-seq-0-c5bc58eeca63f77cbb3ce877c95f42b0-run1/models/step-8294400.model \
    {DATASET_PATH}/network.state

### Model info

In [ ]:
%%file {DATASET_PATH}/network.info
network_name: Classifier
network_settings:
    n_filters: 64

### Input file

In [ ]:
protherm_validaton_dataset_df = (
    pq.read_table("protherm_dataset/protherm_validaton_dataset.parquet")
    .to_pandas()
)

In [ ]:
protherm_validaton_dataset_df.head(2)

In [ ]:
input_dataset_df = (
    protherm_validaton_dataset_df
    .rename(columns={
        "qseq": "sequence",
        "residue_idx_1_corrected": "adjacency_idx_1",
        "residue_idx_2_corrected": "adjacency_idx_2",
    })
    [["sequence", "adjacency_idx_1", "adjacency_idx_2"]]
    .iloc[:10]
)

In [ ]:
pq.write_table(
    pa.Table.from_pandas(input_dataset_df, preserve_index=False),
    DATASET_PATH.joinpath("input.parquet"),
    version="2.0",
    flavor="spark"
)